In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from matplotlib import pyplot as plt
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df=pd.read_csv('../input/tabular-playground-series-may-2022/train.csv',index_col='id')
testdf=pd.read_csv('../input/tabular-playground-series-may-2022/test.csv',index_col='id')

In [ ]:
X=df.drop(['target'],axis=1)
y=df['target']

## Checking and droping Categorial Columns

In [ ]:
#low_cardinality_cols = [cname for cname in X.columns if X[cname].nunique()<100 and X[cname].dtype == 'object']

In [ ]:
#extract object columns
s = (X.dtypes == 'object')
object_cols = list(s[s].index)

In [ ]:
print("Categorical variables:")
print(object_cols)

In [ ]:
X=X.drop(['f_27','f_29'], axis=1)

In [ ]:
X.columns

In [ ]:
X1=X.drop(['f_28'], axis=1)

In [ ]:
X.columns

In [ ]:
X.boxplot(figsize=(40,40))

## Boxplot without column 28

In [ ]:
X1.columns

In [ ]:
X1.boxplot(figsize=(40,40))

In [ ]:
ax = X.plot.hist(density=True, xlim=[0,12], bins=range(1,50))
X.plot.density(ax=ax)
ax.set_xlabel('Density Distribution')

## Removing Outliers

In [ ]:
IQR=X.quantile(0.75)-X.quantile(0.25)
Q1=X.quantile(0.25)
Q3=X.quantile(0.75)

In [ ]:
X = X[~((X < (Q1 - 1.4 * IQR)) |(X > (Q3 + 1.4 * IQR))).any(axis=1)]

In [ ]:
ax = X.plot.hist(density=True, xlim=[0,12], bins=range(1,12))
X.plot.density(ax=ax)
ax.set_xlabel('Density Distribution')

In [ ]:
XIndexList=list(X.index)

In [ ]:
yIndexList=list(y.index)

In [ ]:
y.head()

In [ ]:
y=y.loc[X.index]

## removed outliers

In [ ]:
X.boxplot(figsize=(40,40))

## Since column 28 has dispersed distribution  and  column 14 is skewed ,needs to be transformed


## Column 14 KDE plot

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
# Read CSV file using pandas

# KDE plotting using seaborn
sns.kdeplot(data=X, shade=True, x="f_14", cmap="Blues")
plt.show()

## Column 28 KDE plot (Normal distribution but range is too wide wrt other columns)

In [ ]:
sns.kdeplot(data=X, shade=True, x="f_28", cmap="Blues")
plt.show()

In [ ]:
f14_before=X['f_14'].skew()
f_28_before=X['f_28'].skew()
print(f14_before)
print(f_28_before)

In [ ]:
X['f_14']=np.sqrt(X['f_14'])
X['f_28']=np.sqrt(X['f_28'])

In [ ]:
f14_after=X['f_14'].skew()
f_28_after=X['f_28'].skew()
print("Before Transformation")
print(f14_before)
print(f_28_before)
print("AfterTransformation")
print(f14_after)
print(f_28_after)

In [ ]:
sns.kdeplot(data=X, shade=True, x="f_14", cmap="Blues")
plt.show()

In [ ]:
sns.kdeplot(data=X, shade=True, x="f_28", cmap="Blues")
plt.show()

In [ ]:
X.boxplot(figsize=(40,40))

In [ ]:
X.shape

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_valid, y_train, y_valid = train_test_split(X, y, train_size=0.9, test_size=0.1,
                                                                random_state=0)



In [ ]:
X_train.columns

In [ ]:
X_valid.head()

In [ ]:
#Ordinal Encoding 
from sklearn.preprocessing import OrdinalEncoder
ordinalEncoder=OrdinalEncoder()

label_X_train = X_train.copy()
label_X_valid = X_valid.copy()


In [ ]:
label_X_valid.head()

In [ ]:
#label_X_train[object_cols]=ordinalEncoder.fit_transform(X_train[object_cols])
#label_X_valid[object_cols] = ordinalEncoder.transform(X_valid[object_cols])

In [ ]:
label_X_train.head()


In [ ]:
label_X_valid.head()

# XGBOOST Classifier

In [ ]:
from xgboost import XGBClassifier
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import cross_val_score, KFold

In [ ]:
#xgbc = XGBClassifier()
#print(xgbc)
xgbc=XGBClassifier( booster='gbtree', colsample_bylevel=1,
       colsample_bynode=1, colsample_bytree=1, gamma=0.9, learning_rate=0.003,
       max_delta_step=0, max_depth=15, min_child_weight=1, missing=1,
       n_estimators=2500, n_jobs=-1, nthread=-1,
       objective='binary:logistic', random_state=0, reg_alpha=0.001,
       reg_lambda=0.002, scale_pos_weight=1, seed=None, silent=None,
       subsample=0.8, verbosity=1,eval_metric=["logloss","auc"])

In [ ]:
eval_set=[(y_train, y_valid)]

In [ ]:
xgbc.fit(label_X_train, y_train)

In [ ]:
import xgboost as xgb
xgb.plot_importance(xgbc)
plt.figure(figsize = (16, 12))
plt.show()

# Training and Test Scores

In [ ]:
y_predT = xgbc.predict(label_X_train)
y_predV = xgbc.predict(label_X_valid)


In [ ]:
print("F1 score")
from sklearn.metrics import f1_score
print("Training F1 Score")                                 
print(f1_score(y_predT,y_train, average='weighted'))
print("Validation F1 score")                                  
print(f1_score(y_predV,y_valid, average='weighted'))

In [ ]:
testdf=pd.read_csv('../input/tabular-playground-series-may-2022/test.csv',index_col='id')

In [ ]:
testdf=testdf.drop(['f_27','f_29'], axis=1)
print(testdf.columns)

In [ ]:
testdf['f_14']=np.sqrt(testdf['f_14'])
testdf['f_28']=np.sqrt(testdf['f_28'])

In [ ]:
testdf.head()

In [ ]:
testdf.columns

In [ ]:

X_test_prep = xgbc.predict(testdf)

In [ ]:
X_test_prep

In [ ]:
submission = pd.DataFrame({
  'id' : testdf.index,
   'target' : X_test_prep
})

In [ ]:
submission

In [ ]:
submission.to_csv('./Submission_xgb6.csv', index = False)